## Prediction set up

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.9 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [27]:
# required imports
import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F
import spacy
import tensorflow
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
import io
import os
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
from torchtext.vocab import Vocab

# load spacy module
nlp = spacy.load('en')

# set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import torch
from transformers import AutoModel, AutoTokenizer

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

# For transformers v4.x+:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
class Bertweet_cls(nn.Module):

    def __init__(self, lab2ind, model_path, hidden_size):
        super(Bertweet_cls, self).__init__()
        self.model_path = model_path
        self.hidden_size = hidden_size
        self.bert_model = AutoModel.from_pretrained(model_path, output_hidden_states=True, output_attentions=True)
        
        self.label_num = len(lab2ind)
        
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.hidden_size, self.label_num)

    def forward(self, bert_ids, bert_mask):
        outputs = self.bert_model(input_ids=bert_ids, attention_mask = bert_mask)
        pooler_output = outputs['pooler_output']
        attentions = outputs['attentions']
        
        x = self.dense(pooler_output)
        x = torch.tanh(x)
        x = self.dropout(x)
        fc_output = self.fc(x)

        return fc_output, attentions

In [31]:
lab2ind_sent = {0: 0, 1: 1, 2: 2}
#lab2sent_ind = {0: "Negative", 1: "Neutral", 2: "Positive"}

lab2ind_emoji = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12, 13:13, 14:14, 15:15, 16:16, 17:17, 18:18, 19:19}

In [32]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/ckpt_BERTweet/BERT_sentiment3.pt',map_location=torch.device('cpu'))
model_sentiment = Bertweet_cls(lab2ind_sent, "vinai/bertweet-base", 768).to(device)
model_sentiment.load_state_dict(model['state_dict'])

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [33]:
def sententce_prepocess(content, tokenizer, max_len = 64):
    """
    content: list of string. Each string is a sample. We only include one sample in this list.
    tokenizer: BertTokenizerFast
    """
    #### REF START ####

    # We need to add a special token at the beginning for BERT to work properly.
    content = ["[CLS] " + text for text in content]

    # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
    tokenized_texts = [tokenizer.tokenize(text) for text in content]
    
    # if the sequence is longer the maximal length, we truncate it to the pre-defined maximal length
    tokenized_texts = [ text[:max_len+1] for text in tokenized_texts]

    # We also need to add a special token at the end.
    tokenized_texts = [ text+['[SEP]'] for text in tokenized_texts]
    #print ("Tokenize the first sentence:\n",tokenized_texts[0])
    
    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    #print ("Index numbers of the first sentence:\n",input_ids[0])

    # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
    pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
    input_ids = pad_sequences(input_ids, maxlen=max_len+2, dtype="long", truncating="post", padding="post", value=pad_ind)
    #print ("Index numbers of the first sentence after padding:\n",input_ids[0])

    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for pad tokens
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # Convert all of our data into torch tensors, the required datatype for our model
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)
    #### REF END ####

    return tokenized_texts, inputs, masks

In [34]:
sentence = "When \u2066@nbcsnl\u2069 hits the truth about \u2066@cvspharmacy\u2069 right on the head.\n\nThe only thing longer than a CVS receipt is the time it takes to sort your way through a #Covid vaccine sign-up website. https://t.co/K37bxJdnSl"

In [35]:
def sent_pred(sentence, tokenizer):
  # return the prediction of the given twitter context and the confidence prob
  tokenized_texts, input_ids, masks = sententce_prepocess([sentence], tokenizer)
  input_ids, masks = input_ids.to(device), masks.to(device)
  outputs,_ = model_sentiment(input_ids, masks)
  softmax_fn = nn.Softmax(dim=1)
  outputs = softmax_fn(outputs)
  probabilities, predicted = torch.max(outputs.cpu().data, 1)

  return lab2ind_sent[predicted[0].item()], probabilities

In [40]:
pred, prob = sent_pred(sentence, tokenizer)
print("Sentence = ", sentence)
print("The prediction is: ", pred)
print("Confidence: %.4f" % prob.item())

Sentence =  When ⁦@nbcsnl⁩ hits the truth about ⁦@cvspharmacy⁩ right on the head.

The only thing longer than a CVS receipt is the time it takes to sort your way through a #Covid vaccine sign-up website. https://t.co/K37bxJdnSl
The prediction is:  0
Confidence: 0.6809


## Predicting on mask corpus

In [42]:
masks_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Bertweet/Data/clean_masks_twitter.csv")
masks_t = masks_df["tweet_text"]
masks_df.head(3)

,Unnamed: 0,author_id,created_at,tweet_id,tweet_text,CNN_prediction,CNN_confidence,BERTweet_prediction,BERTweet_confidence,BERTweet_fine-tuned_prediction,BERTweet_fine-tuned_confidence,final_prediction
0,0,835967581693612033,2021-07,1421619622445469697,"The manipulative, politically opportune lying ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,825765275853344773,2021-07,1421619085876547586,@user Someone should introduce a bill mandatin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1412770891335864320,2021-07,1421617357051596804,Just wear a friggin #mask we will never #MakeA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
for id, text in enumerate(masks_t):
  pred, prob = sent_pred(text, tokenizer)
  masks_df.loc[id, "BERTweet_prediction"] = pred
  masks_df.loc[id, "BERTweet_confidence"] = prob.item()

## Predicting on vaccine corpus

In [44]:
vaccine_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Bertweet/Data/clean_vaccines_twitter.csv")
vaccine_t = vaccine_df["tweet_text"]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
for id, text in enumerate(vaccine_t):
  pred, prob = sent_pred(text, tokenizer)
  vaccine_df.loc[id, "BERTweet_prediction"] = pred
  vaccine_df.loc[id, "BERTweet_confidence"] = prob.item()